In [6]:
import cv2
import time
import sys
import numpy as np
from tqdm import tqdm
import os

mainPath = "D:\\Etude\\Doctorat\\Neural Network\\Environment\\yoloOpenCV\\"

# pathToVideo  = mainPath + "input\\Robot_1FPS.mp4"
pathToVideo  = mainPath + "input\\Egocentric_1FPS.mp4"
# pathToONNX   = mainPath + "modelsONNX\\best4.onnx"                # Robot Occluded objects
pathToONNX   = mainPath + "modelsONNX\\egocentric04022022.onnx"
# pathToONNX   = mainPath + "modelsONNX\\robot04022022.onnx"
pathToLabels = mainPath + "labels"

def build_model(is_cuda):
    net = cv2.dnn.readNet(pathToONNX)
    if is_cuda:
        print("Attempty to use CUDA")
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
    else:
        print("Running on CPU")
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
    return net

INPUT_WIDTH = 640
INPUT_HEIGHT = 640
# INPUT_WIDTH = 1920
# INPUT_HEIGHT = 1080

def detect(image, net):
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (640, 640), swapRB=True, crop=False)
    # blob = cv2.dnn.blobFromImage(image, 1/255.0, (640, 640), swapRB=True, crop=False)
    net.setInput(blob)
    preds = net.forward()
    return preds

def load_capture():
    capture = cv2.VideoCapture(pathToVideo)
    return capture

def format_yolov5(frame):
    row, col, _ = frame.shape
    _max = max(col, row)
    result = np.zeros((_max, _max, 3), np.uint8)
    result[0:row, 0:col] = frame
    return result

labelsPath = os.path.sep.join([pathToLabels, "labelsEgocentric.txt"]) # "labelsRobot.txt"]) # "labelsEgocentric.txt"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

is_cuda = len(sys.argv) > 1 and sys.argv[1] == "cuda"
nett = build_model(is_cuda)

def YOLO_OBJECTS(image, frame_w, frame_h, layerOutputs):
    boxes = []
    confidences = []
    classIDs = []
    labels = []
    centPoints = []
    ignoredClasses = ['phone','person','diningtable']
    
    output_data = layerOutputs[0]
    
    rows = output_data.shape[0]
    
    image_width, image_height, _ = image.shape

    x_factor = image_width  / INPUT_WIDTH  #frame_w   # INPUT_WIDTH
    y_factor = image_height / INPUT_WIDTH  #frame_h   # INPUT_HEIGHT
    
    for r in range(rows):
        row = output_data[r]
        confidence = row[4]
        if confidence >= 0.5:
            scores = row[5:]
            _, _, _, max_indx = cv2.minMaxLoc(scores)
            classID = max_indx[1]
            if (scores[classID] > .25):
                confidences.append(confidence)
                classIDs.append(classID)

                x, y, w, h = int(row[0].item()), int(row[1].item()), int(row[2].item()), int(row[3].item())
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    conf = []
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.25)
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # draw a bounding box rectangle and label on the frame
            if  LABELS[classIDs[i]] not in ignoredClasses :
                centPoints.append((int(x+(w/2)) ,int(y+(h/2))))
                # color = [int(c) for c in COLORS[classIDs[i]]]
                # cv2.rectangle(image, (x, y), (x + w, y + h), color, 35)
                # cv2.imwrite(os.path.join(pathOutput, '{}.jpg'.format(uuid.uuid1())), image)
                labels.append(LABELS[classIDs[i]])
                conf.append(format(confidences[i], ".2f"))
                
    indexRHands = []
    indexLHands = []
    indexObject = []
    for xx in range(len(labels)):
        if labels[xx] == 'rhand':
            indexRHands.append(xx) # print("Hooray!!", xx)
        elif labels[xx] == 'lhand':
            indexLHands.append(xx) # print("Hooray!!", xx)
        elif labels[xx] != 'rhand' or labels[xx] != 'lhand':
            indexObject.append(xx)
    return (image, boxes, centPoints, conf, labels, indexRHands, indexLHands, indexObject)


def draw_points (image, pnts, lbls, hands = False):
    image_h, image_w, _ = image.shape
    # print("Type image ", type(image))
    # print("Type pnts ", type(pnts))
    # print("Type lbls ", type(lbls))
    for idx, pnt in enumerate(pnts):
        color = []
        if hands:
            color = [0,255,0]
        else:
            color = [0,0,255]
        cv2.circle(image, (pnt[0], pnt[1]), 8, color , -1 , 8)
        cv2.line(image, (pnt[0] + 16, pnt[1] - 15), (pnt[0] + 50, pnt[1] - 30), [100, 10, 255], 2, 8)
        cv2.putText(image, lbls[idx] , (pnt[0] + 50, pnt[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, [50, 200, 250], 2, cv2.LINE_AA)
    return image

Running on CPU


In [2]:
################################################
# Ontology Reasoning Functions
################################################

def load_data():
    model = KeyedVectors.load_word2vec_format(mainPath + "modelsONNX\\numberbatch-en.txt")
    return model

def remove_duplicate(items):
  return list(dict.fromkeys(items))


def get_Affordances(model, totalObjects, objectsInAction, activities):
    affordances = []
    affordancesInAction = []

    totalObjects = [obj.lower() for obj in totalObjects]
    objectsInAction = [obj.lower() for obj in objectsInAction]

    for obj in totalObjects :

        affords = model.most_similar_to_given(obj,activities)
        affordances.append(affords)
        # print('Object : {0} affords : {1}'.format(obj,affords))

    for obj in objectsInAction:
        affordancesInAction.append(affordances[totalObjects.index(obj)])

    affordances = remove_duplicate(affordances)
    affordancesInAction = remove_duplicate(affordancesInAction)
    return affordances,affordancesInAction

def mapObjectsToConceptNet(objList):
    ConceptNetMap = {
        'pottedplant' : 'plant',
        'diningtable' : 'table',
        'tvmonitor' : 'tv',
        'traffic light' : 'light',
        'cell phone' : 'phone',
        'right hand' : 'rhand',
        'left hand' : 'lhand'
    }

    for idx, obj in enumerate(objList):
        if obj in ConceptNetMap.keys():
            objList[idx] = ConceptNetMap[obj]
    return objList

In [3]:
import uuid
from scipy import spatial
from gensim.models import KeyedVectors
from gensim.downloader import base_dir

# Define Main Activities
activities = ['eat','drink','rest','socialize','work','write']

ConceptNetmodel = load_data()
print ("ConceptNet Model Loaded ....")

ConceptNet Model Loaded ....


In [4]:
import os
import re
import random

def buildAffordanceLeft(pLoc, pLocPr, cLoc, cLocPr, pOoi, pOoiPr, cOoi, cOoiPr):
    # File One: Read the sorts and store in a .txt file.
    pLocation = 'location(' + pLoc + ',' + ' ' + pLocPr + ').'
    cLocation = 'location(' + cLoc + ',' + ' ' + cLocPr + ').'
    pObjectOI = 'objectOI(' + pOoi + ',' + ' ' + pOoiPr + ').'
    cObjectOI = 'objectOI(' + cOoi + ',' + ' ' + cOoiPr + ').'
    
    # print (pLocation)
    # print (cLocation)
    # print (pObjectOI)
    # print (cObjectOI)
    
    with open('./lp/sortsLeft.lp', 'w') as out:
        out.write('{}\n{}\n{}\n{}\n'.format(pLocation, cLocation, pObjectOI, cObjectOI))
        # print ('\n./lp/sortsLeft.lp is stored')
    
    sortsLeft = open("./lp/sortsLeft.lp")
    linesortsLeft = sortsLeft.readlines()
        
    # File Two: Read the sorts, build the events and store in a .txt file.
    pEChanLoc = '\nhappens(changeOOI(' + pOoi + ',' + ' ' + cOoi + '), 1).'
    pEChanOOI = 'happens(changeLocation(' + pLoc + ',' + ' ' + cLoc + '), 1).'

    # print (pEChanLoc)
    # print (pEChanOOI)

    csloi = "not releasedAt(F,0) :- fluent(F)."

    with open('./lp/eventsLeft.lp', 'w') as out:
        out.write('{}\n{}\n{}\n'.format(csloi, pEChanLoc, pEChanOOI))
        # print ('\n./lp/eventsLeft.lp is stored')
    
    eventsLeft = open("./lp/eventsLeft.lp")
    lineeventsLeft = eventsLeft.readlines()
    
    command = "clingo\\clingoWindows.exe -c maxtime=3 -n 1 " +\
              "lp\\decOrigin2.lp " +\
              "lp\\sortsLeft.lp " +\
              "lp\\eventsLeft.lp " +\
              "lp\\activities.lp " +\
              "lp\\ecasp888.lp " +\
              "lp\\ecasp88888.lp | " +\
              "clingo\\format-output.exe 3 > " +\
              "inference\\ecasp88Left.txt"
    
    # command = "./clingo/clingoLinux -c maxtime=3 -n 1 ./lp/decOrigin2.lp ./lp/sorts.lp ./lp/events.lp ./lp/activities.lp ./lp/ecasp888.lp ./lp/ecasp88888.lp | ./clingo/format-output 3 > ./inference/ecasp88.txt"

    os.system(command)
    
    intention = open(mainPath + "inference\\ecasp88Left.txt")
    line = intention.readlines()
    intention = re.search('\+intention\((.+?),', line[15])
    if intention:
        foundIntenLeft = intention.group(1)
        foundConfLeft = line[15][-4:-2]
    else:
        foundIntenLeft = []
        foundConfLeft = []
    # print("\nfoundIntenLeft:", foundIntenLeft)
    # print("foundConf:", foundConfLeft)
    return foundIntenLeft, foundConfLeft

def buildAffordanceRight(pLoc, pLocPr, cLoc, cLocPr, pOoi, pOoiPr, cOoi, cOoiPr):
    # File One: Read the sorts and store in a .txt file.
    pLocation = 'location(' + pLoc + ',' + ' ' + pLocPr + ').'
    cLocation = 'location(' + cLoc + ',' + ' ' + cLocPr + ').'
    pObjectOI = 'objectOI(' + pOoi + ',' + ' ' + pOoiPr + ').'
    cObjectOI = 'objectOI(' + cOoi + ',' + ' ' + cOoiPr + ').'
    
    # print (pLocation)
    # print (cLocation)
    # print (pObjectOI)
    # print (cObjectOI)
    
    with open('./lp/sortsRight.lp', 'w') as out:
        out.write('{}\n{}\n{}\n{}\n'.format(pLocation, cLocation, pObjectOI, cObjectOI))
        # print ('\n./lp/sortsRight.lp is stored')
        
    # File Two: Read the sorts, build the events and store in a .txt file.
    pEChanLoc = '\nhappens(changeOOI(' + pOoi + ',' + ' ' + cOoi + '), 1).'
    pEChanOOI = 'happens(changeLocation(' + pLoc + ',' + ' ' + cLoc + '), 1).'

    # print (pEChanLoc)
    # print (pEChanOOI)

    csloi = "not releasedAt(F,0) :- fluent(F)."

    with open('./lp/eventsRight.lp', 'w') as out:
        out.write('{}\n{}\n{}\n'.format(csloi, pEChanLoc, pEChanOOI))
        # print ('\n./lp/eventsRight.lp is stored')
    
    command = "clingo\\clingoWindows.exe -c maxtime=3 -n 1 " + \
              "lp\\decOrigin2.lp " +\
              "lp\\sortsRight.lp " +\
              "lp\\eventsRight.lp " +\
              "lp\\activities.lp " +\
              "lp\\ecasp888.lp " +\
              "lp\\ecasp88888.lp | " +\
              "clingo\\format-output.exe 3 > " +\
              "inference\\ecasp88Right.txt"
    # command = "./clingo/clingoLinux -c maxtime=3 -n 1 ./lp/decOrigin2.lp ./lp/sorts.lp ./lp/events.lp ./lp/activities.lp ./lp/ecasp888.lp ./lp/ecasp88888.lp | ./clingo/format-output 3 > ./inference/ecasp88.txt"

    os.system(command)
    
    intention = open(mainPath + "inference\\ecasp88Right.txt")
    line = intention.readlines()
    intention = re.search('\+intention\((.+?),', line[15])
    if intention:
        foundIntenRight = intention.group(1)
        foundConfRight = line[15][-4:-2]
    else:
        foundIntenRight = []
        foundConfRight = []
    # print("\nfoundIntenRight:", foundIntenRight)
    # print("foundConfRight:", foundConfRight)
    return foundIntenRight, foundConfRight

def sceneDetection():
    x = random.uniform(0.65,0.95)
    x = format(x, ".2f")
    x = str(x)
    x = x[2:4]
    # print(x)
    return x

In [7]:
###############################
#   Predict bounding boxes
###############################

video_reader = cv2.VideoCapture(pathToVideo)

nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
frame_h   = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_w   = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))

#INPUT_WIDTH = frame_w
#INPUT_HEIGHT = frame_h

video_out = pathToVideo[:-4] + '_detected' + pathToVideo[-4:]

video_writer = cv2.VideoWriter(video_out,
                       cv2.VideoWriter_fourcc(*'XVID'), 
                       50.0,
                       (frame_h, frame_h))

cv2.namedWindow("Image", cv2.WINDOW_NORMAL)

objectsOfIntrestLEC = []
objectsOfIntrestLECConf = []
objectsOfIntrestREC = []
objectsOfIntrestRECConf = []

foundIntenLeft = "init"
foundConfLeft = "init"
foundIntenRight = "init"
foundConfRight = "init"

for i in tqdm(range(nb_frames)):
    # Read Image
    _, image = video_reader.read()
    # blank_image = np.zeros((frame_h,frame_w,3), np.uint8)
    # Recognize Objects
    
    image = cv2.resize(image, (1080, 1080))
    
    image = format_yolov5(image)
    
    layerOutputs = detect(image, nett)
    
    image, OBJboxes, OBJcentPoints, confidences, labels, indexRHands, indexLHands, \
                indexObject = YOLO_OBJECTS(image, frame_w, frame_h, layerOutputs)
    
    # Recognize Hands
    OBJECTcentPoints = []
    OBJECTlabels     = []
    confObjects      = []
    RHANDcentPoints  = []
    RHANDlabels      = []
    confRHand        = []
    LHANDcentPoints  = []
    LHANDlabels      = []
    confLHand        = []
    
    for oo in range(len(indexObject)):
        OBJECTcentPoints.append(OBJcentPoints[indexObject[oo]])
        OBJECTlabels.append(labels[indexObject[oo]])
        confObjects.append(confidences[indexObject[oo]])
        
    for rr in range(len(indexRHands)):
        RHANDcentPoints.append(OBJcentPoints[indexRHands[rr]])
        RHANDlabels.append(labels[indexRHands[rr]])
        confRHand.append(confidences[indexRHands[rr]])
        
    for ll in range(len(indexLHands)):
        LHANDcentPoints.append(OBJcentPoints[indexLHands[ll]])
        LHANDlabels.append(labels[indexLHands[ll]])
        confLHand.append(confidences[indexLHands[ll]])
        
    OBJECTcentPoints = list(OBJECTcentPoints)
    RHANDcentPoints  = list(RHANDcentPoints)
    LHANDcentPoints  = list(LHANDcentPoints)
    
    if len(OBJECTcentPoints) > 0:
        image = draw_points(image, OBJECTcentPoints[0:8], OBJECTlabels[0:8], hands=False)
        
    if len(LHANDcentPoints) > 0:
        image = draw_points(image, LHANDcentPoints, LHANDlabels, hands=True)
        
    if len(RHANDcentPoints) > 0:
        image = draw_points(image, RHANDcentPoints, RHANDlabels, hands=True)

    # Calculate nearest neigbour
    objectsOfIntrestR = []
    objectsOfIntrestL = []
    
    if len(OBJECTcentPoints) > 0:
        tree = spatial.KDTree(OBJECTcentPoints)
        for lhand in LHANDcentPoints :
            distance, LnearestOBJidx = tree.query(lhand)
            LnearestOBJidx = int(LnearestOBJidx)
            l = tuple(tree.data[LnearestOBJidx])
            idxl = OBJECTcentPoints.index(tuple(tree.data[LnearestOBJidx]))
            objectsOfIntrestL.append(OBJECTlabels[idxl])
            """print("\nconfObjects[idxl]", confObjects[idxl])
            print("confObjects[idxl] type", type(confObjects[idxl]))
            print("len(confObjects[idxl])", len(confObjects[idxl]))
            if confObjects[idxl] != []:
                objectsOfIntrestLECConf.append(confObjects[idxl])
            elif objectsOfIntrestL == []:
                objectsOfIntrestLECConf = objectsOfIntrestLECConf"""
            cv2.line(image, lhand, (int(l[0]),int(l[1])), (255,0,0), 5)
            
        for rhand in RHANDcentPoints:
            distance, RnearestOBJidx = tree.query(rhand)
            RnearestOBJidx = int(RnearestOBJidx)
            r = tuple(tree.data[RnearestOBJidx])
            idxr = OBJECTcentPoints.index(tuple(tree.data[RnearestOBJidx]))
            objectsOfIntrestR.append(OBJECTlabels[idxr])
            """print("\nconfObjects[idxl]", confObjects[idxl])
            print("confObjects[idxl] type", type(confObjects[idxl]))
            print("len(confObjects[idxl])", len(confObjects[idxl]))
            if confObjects[idxl] != []:
                objectsOfIntrestRECConf.append(confObjects[idxl])
            elif objectsOfIntrestL == []:
                objectsOfIntrestRECConf = objectsOfIntrestRECConf"""
            cv2.line(image, rhand, (int(r[0]),int(r[1])), (255,0,0), 5)
            
    if len(OBJECTlabels) > 0:
        totalObjects = mapObjectsToConceptNet(OBJECTlabels)
    else:
        totalObjects = []

    cv2.putText(image, 'Objs: {}'.format(totalObjects[0:8]) , (15,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0], 2, cv2.LINE_AA)
    # cv2.putText(image, 'Objs Conf.: {}'.format(confObjects[0:7]) , (25,65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0], 2, cv2.LINE_AA)
    
    if len(objectsOfIntrestR) > 0:
        objectsInActionR = mapObjectsToConceptNet(objectsOfIntrestR)
    else:
        objectsInActionR = []
        
    if len(objectsOfIntrestL) > 0:
        objectsInActionL = mapObjectsToConceptNet(objectsOfIntrestL)
    else:
        objectsInActionL = []

    cv2.putText(image, 'Objs Int: LHand {} - RHand {}'.format(objectsInActionL, objectsInActionR) , (25,65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0],2,cv2.LINE_AA)
    # cv2.putText(image, 'Objs Int: LHand {}{} - RHand {}{}'.format(objectsOfIntrestL, confLHand, objectsOfIntrestR, confRHand) , (15,65), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0],2,cv2.LINE_AA)
    # cv2.putText(image, 'Objs Int Conf.: LHand {} - RHand {}'.format(confLHand, confRHand) , (25,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 255, 0],2,cv2.LINE_AA)
        
    if objectsOfIntrestL != []:
        objectsOfIntrestLEC.append(objectsOfIntrestL[0])
    elif objectsOfIntrestL == []:
        objectsOfIntrestLEC = objectsOfIntrestLEC
        
    if objectsOfIntrestR != []:
        objectsOfIntrestREC.append(objectsOfIntrestR[0])
    elif objectsOfIntrestR == []:
        objectsOfIntrestREC = objectsOfIntrestREC
    
    pLoc   = 'office'
    pLocPr = sceneDetection()
    cLoc   = 'desktop'
    cLocPr = sceneDetection()
    
    if len(objectsOfIntrestLEC) > 2:
        if objectsOfIntrestLEC[-1] == objectsOfIntrestLEC[-2]:
            objectsOfIntrestLEC[-1] = objectsOfIntrestLEC[-1] + '2'
        pOoiLeft   = objectsOfIntrestLEC[-2]
        pOoiPrLeft = confidences[-2]
        pOoiPrLeft = str(pOoiPrLeft)
        pOoiPrLeft = pOoiPrLeft[2:4]
        cOoiLeft   = objectsOfIntrestLEC[-1]
        cOoiPrLeft = confidences[-2]
        cOoiPrLeft = str(cOoiPrLeft)
        cOoiPrLeft = cOoiPrLeft[2:4]
        # print("\nobjectsOfIntrestLEC[-1]", objectsOfIntrestLEC[-1])
        # print("objectsOfIntrestLEC[-2]", objectsOfIntrestLEC[-2])
        foundIntenLeft, foundConfLeft = buildAffordanceLeft(pLoc, pLocPr, cLoc, cLocPr, pOoiLeft, pOoiPrLeft, cOoiLeft, cOoiPrLeft)
    
    if len(objectsOfIntrestREC) > 2:
        if objectsOfIntrestREC[-1] == objectsOfIntrestREC[-2]:
            objectsOfIntrestREC[-1] = objectsOfIntrestREC[-1] + '2'
        # print("\nobjectsOfIntrestREC[-1]", objectsOfIntrestREC[-1])
        # print("objectsOfIntrestREC[-2]", objectsOfIntrestREC[-2])
        pOoiRight   = objectsOfIntrestREC[-2]
        pOoiPrRight = confidences[-2]
        pOoiPrRight = str(pOoiPrRight)
        pOoiPrRight = pOoiPrRight[2:4]
        cOoiRight   = objectsOfIntrestREC[-1]
        cOoiPrRight = confidences[-1]
        cOoiPrRight = str(cOoiPrRight)
        cOoiPrRight = cOoiPrRight[2:4]
        foundIntenRight, foundConfRight = buildAffordanceRight(pLoc, pLocPr, cLoc, cLocPr, pOoiRight, pOoiPrRight, cOoiRight, cOoiPrRight)
        
    try:
        affordancesR , affInActionR = get_Affordances(ConceptNetmodel,totalObjects,objectsInActionR,activities)
        affordancesL , affInActionL = get_Affordances(ConceptNetmodel,totalObjects,objectsInActionL,activities)
    except Exception as e:
        print("errrerere", e)
        affordancesR , affInActionR = [], []
        affordancesL , affInActionL = [], []
    # cv2.putText(blank_image, 'Affordances : {0}'.format(affordances) , (50,300), cv2.FONT_HERSHEY_SIMPLEX, 2, [255, 0, 0],3,cv2.LINE_AA)
    cv2.putText(image, 'Affordances: LHand {} - RHand {}'.format(affordancesL[0:3], affordancesR[0:3]) , (15,100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [255, 0, 0],2,cv2.LINE_AA)
    # cv2.putText(blank_image, 'Activities : {0}'.format(affInAction) , (50,400), cv2.FONT_HERSHEY_SIMPLEX, 2, [0, 0, 255],3,cv2.LINE_AA)
    # cv2.putText(image, 'Activities:  LHand {} - RHand {}'.format(affInActionL, affInActionR) , (25,170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
    
    cv2.putText(image, 'Activities: LHand {} {} - RHand {} {}'.format("['"+foundIntenLeft+",",foundConfLeft+"']",\
                                                                      "['"+foundIntenRight+",",foundConfRight+"']"),\
                                                        (15,135), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0, 0, 255],2,cv2.LINE_AA)
    
    cv2.imshow('Image', image)
    
    # Write Video Frames
    # pathOutputFramesRobot = mainPath + "outputFramesRobot"
    pathOutputFramesRobot = mainPath + "outputFramesEgocentric"
    # vis = np.concatenate((image,blank_image), axis=0)
    # vis = cv2.resize(vis,None,fx=0.5,fy=0.5)
    key = cv2.waitKey(30)
    cv2.imwrite(os.path.join(pathOutputFramesRobot, '{}.jpg'.format(uuid.uuid1())), image)
    video_writer.write(np.uint8(image))

100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [02:32<00:00,  1.84s/it]


In [8]:
video_reader.release()
video_writer.release()
cv2.destroyAllWindows()

In [9]:
# Resize video

import cv2
import numpy as np

pathToResize = mainPath + "input\\"

# cap = cv2.VideoCapture(pathToResize + "Robot_1FPS_detected.mp4")
cap = cv2.VideoCapture(pathToResize + "Egocentric_1FPS_detected.mp4")
 
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(pathToResize + "Robot_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
out = cv2.VideoWriter(pathToResize + "Egocentric_1FPS_detectedResized.mp4",fourcc, 3.0, (1920,1080))
 
while True:
    ret, frame = cap.read()
    if ret == True:
        b = cv2.resize(frame,(1920,1080))
        out.write(b)
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import sys
import fileinput

def replacement(file):
    for line in fileinput.input(file, inplace=1, backup='.bak'):
        if "0 " in line[0:2]:
            line = line[0:2].replace("0 ","80 ") + line[2:]
        elif "1 " in line[0:2]:
            line = line[0:2].replace("1 ","81 ") + line[2:]
        elif "2 " in line[0:2]:
            line = line[0:2].replace("2 ","82 ") + line[2:]
        elif "3 " in line[0:2]:
            line = line[0:2].replace("3 ","83 ") + line[2:]
        elif "4 " in line[0:2]:
            line = line[0:2].replace("4 ","84 ") + line[2:]
        elif "5 " in line[0:2]:
            line = line[0:2].replace("5 ","85 ") + line[2:]
        elif "6 " in line[0:2]:
            line = line[0:2].replace("6 ","86 ") + line[2:]
        elif "7 " in line[0:2]:
            line = line[0:2].replace("7 ","87 ") + line[2:]
        sys.stdout.write(line)
    fileinput.close()

for i in range(54):
    file = "D:\\Etude\\Doctorat\\VirtualMachines\\sharedFolder\\ModifiedOpenLabelling-main2\\bbox_txt\\frame" + str(i) + ".txt"
    replacement(file)

In [ ]:
# Reduce FPS

import cv2

path = "D:\\Etude\\Doctorat\\Inspiration Anticipation-Prediction\\Code\\input\\Egocentric.mp4"
pathOutputFrames = "D:\\Etude\\Doctorat\\Inspiration Anticipation-Prediction\\Code\\yolov5-master\\yoloOpenCV\\outputFramesEgocentric\\frame"

vidcap = cv2.VideoCapture(path)

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000) 
    hasFrames,image = vidcap.read() 
    if hasFrames:
        cv2.imwrite(pathOutputFrames + str(sec) + ".jpg", image)
    return hasFrames 

sec = 0
frameRate = 1 #it will capture image in each 0.5 second
success = getFrame(sec)
while success:
    sec = sec + frameRate 
    sec = round(sec, 2) 
    success = getFrame(sec)

In [ ]:
# Frames Building

import cv2

pathInputFrames = "D:\\Etude\\Doctorat\\Inspiration Anticipation-Prediction\\Code\\yolov5-master\\yoloOpenCV\\outputFrames\\frame"
pathOutputVideo = "D:\\Etude\\Doctorat\\Inspiration Anticipation-Prediction\\Code\\input\\Robot_1FPS.mp4"

img_array = []

def toCheck(frame):
    x = 0
    if frame is None:
        return None

def openFrame(x):
    path = pathInputFrames + str(x) + ".jpg"
    frame = cv2.imread(path)
    return frame
    
for x in range(1998):
    frame = openFrame(x)
    if frame is None:
        x += 1
        toCheck(frame)
    else:   
        height, width, layers = frame.shape
        size = (width, height)
        img_array.append(frame)

        output = cv2.VideoWriter(pathOutputVideo, cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    output.write(img_array[i])
output.release()